## Rannable Object Binding Tools

In [19]:
from pprint import pprint
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from chat_model_client import get_model
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import (
    RunnablePassthrough
)

1. Tools声明

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "solver",
            "description": "表述并解一个方程",
            "parameters": {
                "type": "object",
                "properties": {
                    "equation": {
                        "type": "string",
                        "description": "待解的数学表达式"
                    },
                    "solution": {
                        "type": "string",
                        "description": "方程的解"
                    }
                },
                "required": ["equation", "solution"]
            }
        }
        
    }
]

2. 声明返回结构

In [21]:
class Solver(BaseModel):
    equation: str = Field(
        description="数学表达式"
    ),
    solution: str = Field(
        description="方程的解"
    )

2. 绑定tools

In [22]:
query = "x的3次方加上11等于75"
parser = PydanticOutputParser(pydantic_object=Solver)
prompt = PromptTemplate(
    template="你是一个乐于助人的助手。写出下面的方程，然后求解。\n{format_instructions}\n{query}\n如果输出的是代码块，请不要包括首尾的```符号",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# prompt = ChatPromptTemplate([
#     ("system", "你是一个乐于助人的助手。写出下面的方程，然后求解"),
#     ("human", "{input}")
# ])

llm = get_model('deepseek')
llm.bind_tools(tools=tools)

/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='数学表达式'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x136366d40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x136384730>, root_client=<openai.OpenAI object at 0x136367250>, root_async_client=<openai.AsyncOpenAI object at 0x136366dd0>, model_name='deepseek-chat', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.deepseek.com', max_tokens=2000), kwargs={'tools': [{'type': 'function', 'function': {'name': 'solver', 'description': '表述并解一个方程', 'parameters': {'type': 'object', 'properties': {'equation': {'type': 'string', 'description': '待解的数学表达式'}, 'solution': {'type': 'string', 'description': '方程的解'}}, 'required': ['equation', 'solution']}}}]}, config={}, config_factories=[])

3. 构建并执行runnable

In [23]:

chain = {"query": RunnablePassthrough()} | prompt | llm | parser
output = chain.invoke({"query": query})

pprint(output)

Solver(equation='x^3 + 11 = 75', solution='x = 4')


4. 链路检查

In [24]:
pprint(chain.get_graph().print_ascii())

+----------------------+ 
| Parallel<query>Input | 
+----------------------+ 
            *            
            *            
            *            
    +-------------+      
    | Passthrough |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOpenAI |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Solver |        
       +--------+        
None
